In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_dataset,mnist_info=tfds.load(name='mnist',as_supervised=True,with_info=True)
type(mnist_dataset)

dict

In [3]:
m_train,m_test=mnist_dataset['train'],mnist_dataset['test']
validation_samples=0.1 * mnist_info.splits['train'].num_examples
test_samples= mnist_info.splits['test'].num_examples
validation_samples=tf.cast(validation_samples,tf.int64) # validation samples could be float so we cast it to int

def scale(image,label):
  image=tf.cast(image,tf.float32)
  image/=255. # So that the values remains between 0 and 1
  return image, label

scaled_train_and_validation_data= m_train.map(scale)
scaled_test_data=m_test.map(scale)

shuffled_train_and_val_data= scaled_train_and_validation_data.shuffle(10000) # we don't shuffe whole data at once to prevent memory overfow

val_data= shuffled_train_and_val_data.take(validation_samples)
train_data= shuffled_train_and_val_data.skip(validation_samples)

type(train_data)

tensorflow.python.data.ops.skip_op._SkipDataset

In [4]:
train_data=train_data.batch(100) #SGD
val_data= val_data.batch(validation_samples)
test_data= m_test.batch(test_samples)

In [5]:
validation_inputs, validation_targets = next(iter(val_data))

# Model

In [22]:
input_size=784
output_size=10
hidden_layer_size=1000

In [26]:
model=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='tanh'), # sigmoid gave lower accuracy
    tf.keras.layers.Dense(output_size,activation='softmax'),
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [27]:
EPOCHS=10
model.fit(train_data,epochs=EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/10
540/540 - 5s - loss: 0.2006 - accuracy: 0.9377 - val_loss: 0.1224 - val_accuracy: 0.9595 - 5s/epoch - 10ms/step
Epoch 2/10
540/540 - 3s - loss: 0.0865 - accuracy: 0.9736 - val_loss: 0.0708 - val_accuracy: 0.9777 - 3s/epoch - 6ms/step
Epoch 3/10
540/540 - 4s - loss: 0.0601 - accuracy: 0.9814 - val_loss: 0.0648 - val_accuracy: 0.9815 - 4s/epoch - 7ms/step
Epoch 4/10
540/540 - 3s - loss: 0.0466 - accuracy: 0.9855 - val_loss: 0.0552 - val_accuracy: 0.9853 - 3s/epoch - 6ms/step
Epoch 5/10
540/540 - 3s - loss: 0.0390 - accuracy: 0.9879 - val_loss: 0.0489 - val_accuracy: 0.9867 - 3s/epoch - 6ms/step
Epoch 6/10
540/540 - 4s - loss: 0.0363 - accuracy: 0.9888 - val_loss: 0.0343 - val_accuracy: 0.9888 - 4s/epoch - 8ms/step
Epoch 7/10
540/540 - 3s - loss: 0.0308 - accuracy: 0.9899 - val_loss: 0.0381 - val_accuracy: 0.9892 - 3s/epoch - 6ms/step
Epoch 8/10
540/540 - 3s - loss: 0.0289 - accuracy: 0.9917 - val_loss: 0.0411 - val_accuracy: 0.9888 - 3s/epoch - 6ms/step
Epoch 9/10
540/540 - 4s

In [32]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 77ms/step - loss: 0.1466 - accuracy: 0.9784


In [29]:
print(f'Test Loss:{test_loss:.2f}, Test Accuracy:{test_accuracy*100:.2f}%') # 250,5,10

Test Loss:0.15, Test Accuracy:97.84%


1. using tanh activation function didn't help in increasing  accuracy. However, when we increase depth and width to 3, 400 and use 10 epochs, the model higher accuracy